 # Problema de Regresión

Caso de estudio, instituciones educativas y numero de estudiantes

Importamos CSV

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Cargar datos
df = pd.read_csv('estudios.csv', sep=';', quotechar='"', encoding='utf-8', engine='python')

# Preprocesamiento
df['Anio'] = df['Anio_lectivo'].str.extract(r'(\d{4})').astype(int)
df = pd.get_dummies(df, columns=['Sostenimiento', 'Area', 'Regimen_Escolar'])


In [ ]:
df.head()
list(df.columns)
df.drop(columns=['Anio_lectivo', 'Zona','Nombre_Institucion'], inplace=True)
#list(df.columns)

In [ ]:
df.head()
list(df.columns)

## EDA

In [ ]:
# Variables
X = df[['Anio', 'Total_Docentes', 'Sostenimiento_Fiscal', 'Area_Rural']]  # por ejemplo
y = df['Total_Estudiantes']

# Modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Predicción
preds = modelo.predict(X_test)

## Split, Pipeline y fit

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import altair as alt
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipe = Pipeline([
    ("escalado", StandardScaler()),
    ("lineal", LinearRegression())
])


# 7. Entrenar
pipe.fit(X_train, y_train)

# 8. Predecir
y_pred_pipe = pipe.predict(X_test)

 ### Evaluacion

In [ ]:
rmse = mean_squared_error(y_test, y_pred_pipe)**0.5
r2 = r2_score(y_test, y_pred_pipe)

print("RMSE:", rmse)
mae = mean_absolute_error(y_test, y_pred_pipe)
print("Mae:", mae)
print("R²:", r2)

Graficos

In [ ]:
alt.data_transformers.disable_max_rows()

df_vp = pd.DataFrame({
    'Valores_Reales': y_test,
    'Predicciones': y_pred_pipe
})

# Gráfico scatter plot: Valores reales vs predicciones
scatter = alt.Chart(df_vp).mark_circle(size=60, opacity=0.6).encode(
    x='Valores_Reales',
    y='Predicciones',
    tooltip=['Valores_Reales', 'Predicciones']
)

alt.Chart(df_vp).mark_circle(size=60, opacity=0.6).encode(
    x='Valores_Reales',
    y='Predicciones',
    tooltip=['Valores_Reales', 'Predicciones']
)

line = alt.Chart(pd.DataFrame({
    'x': [df_vp['Valores_Reales'].min(), df_vp['Valores_Reales'].max()]
})).mark_line(color='red').encode(
    x='x',
    y='x'
)

alt.Chart(pd.DataFrame({
    'x': [df_vp['Valores_Reales'].min(), df_vp['Valores_Reales'].max()]
})).mark_line(color='red').encode(
    x='x',
    y='x'
)


scatter_plot = (scatter + line).properties(
    width=600,
    height=400,
    title='Predicciones vs Valores Reales'
)
scatter | line | scatter_plot
# Gráfico histograma de errores
df_errores = pd.DataFrame({'Error': df_vp['Valores_Reales'] - df_vp['Predicciones']})

histograma = alt.Chart(df_errores).mark_bar().encode(
    alt.X('Error', bin=alt.Bin(maxbins=30)),
    y='count()'
).properties(
    width=600,
    height=300,
    title='Distribución de Errores'
)


In [ ]:
histograma

In [ ]:
scatter_plot

### Yachay Tech